In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

from lib import Models, Datasets, LossCalculator, Train


2025-02-22 00:06:49.990390: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 00:06:50.004328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740172010.021114  397301 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740172010.026359  397301 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 00:06:50.044488: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# model, tokenizer = Models.LoadLLM("meta-llama/Llama-3.2-1B-Instruct")
model, tokenizer = Models.LoadLLMLoRA("./logs/models/step1")

CR_train = Datasets.LoadCommonReasoning('train', 3000)

print(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features=2048,

In [3]:
CR_valid = Datasets.LoadCommonReasoning('validation', 3000)

print(CR_valid)

LossCalculator.CalcLoss(**CR_valid, max_length=1024, model=model, tokenizer=tokenizer)

{'dataset': Dataset({
    features: ['text', 'text_wa_answer', 'correct_answer', 'task'],
    num_rows: 3000
}), 'name_text': 'text'}


Running tokenizer on dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

/home/parshina/workspace/AdaptorsResearch/lib/LossCalculator.py:98: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'eval_loss': 0.5812273025512695,
 'eval_model_preparation_time': 0.008,
 'eval_accuracy': 0.5683333333333372,
 'eval_runtime': 629.7757,
 'eval_samples_per_second': 4.764,
 'eval_steps_per_second': 1.191}

In [4]:
# from peft import LoraConfig, TaskType, get_peft_model

# peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj'])
# # peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['c_attn', 'c_proj', 'c_fc', 'c_proj'])


# model['model'] = get_peft_model(model['model'], peft_config)

# print(model['model'])

model = Models.PrepareLoRA(model, 8, 16, 0.05)


In [5]:
Train.Trains(**CR_train, max_length=1024, model=model, tokenizer=tokenizer)

Running tokenizer on dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

/home/parshina/workspace/AdaptorsResearch/lib/Train.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.420800
20,0.792100
30,0.603300
40,0.611300
50,0.570600
60,0.567300
70,0.595200
80,0.596100
90,0.523100


TrainOutput(global_step=93, training_loss=0.691883481958861, metrics={'train_runtime': 1458.8314, 'train_samples_per_second': 2.056, 'train_steps_per_second': 0.064, 'total_flos': 1.7872620431081472e+16, 'train_loss': 0.691883481958861, 'epoch': 0.992})

In [6]:
# LossCalculator.CalcLoss(**math, max_length=1024, count=100, model=model['model'], tokenizer=model['tokenizer'])

# model_path = "./logs/test_new/checkpoint-200"  # Путь к папке с сохраненной моделью
# from transformers import AutoTokenizer, AutoModelForCausalLM

# t = AutoTokenizer.from_pretrained(model_path)
# m = AutoModelForCausalLM.from_pretrained(model_path)

In [7]:
from transformers import pipeline


dirc = "logs/cats_model/checkpoint-150"
# model, tokenizer = Models.LoadLLMLoRA(dirc)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
# generator = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")



# messages = [
#     {"role": "user", "content": "Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\nCompute $\dbinom{16}{5}$."},
# ]

messages = [
    {"role": "system", "content": "Please answer the following question with True or False. Follow the answer format, full answer not needed."},
    {"role": "user", "content": "Question: do iran and afghanistan speak the same language\nAnswer format: True/False"},
]



generated_text = generator(messages,  max_length=500, num_return_sequences=1)

for i, text in enumerate(generated_text):
    print(f"Generated text {i + 1}:")
    text = text['generated_text']
    for entry in text:
        if entry['role'] == 'assistant':
            print(entry['content'])


Device set to use cuda
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jamba

Generated text 1:
True


In [8]:
messages = [
    {"role": "system", "content": "Please answer the following question with True or False. Follow the answer format, full answer not needed."},
    {"role": "user", "content": "Question: do good samaritan laws protect those who help at an accident\nAnswer format: True/False"},
]



generated_text = generator(messages,  max_length=500, num_return_sequences=1)

for i, text in enumerate(generated_text):
    print(f"Generated text {i + 1}:")
    text = text['generated_text']
    for entry in text:
        if entry['role'] == 'assistant':
            print(entry['content'])

Generated text 1:
True. Good Samaritan laws typically protect those who help at an accident or emergency from being held liable for damages or injuries they may cause. These laws often provide immunity from civil lawsuits for acts of kindness, such as providing medical care or assistance to someone in distress.


In [9]:
messages = [
    {"role": "system", "content": "Please answer the following question with True or False. Follow the answer format, full answer not needed."},
    {"role": "user", "content":

"""Health. [header] How to identify performance anxiety [title] Determine if your pulse is elevated. [step] When you are anxious about a performance, your pulse will become elevated. This is your body's response to the stress and worry of your impending performance.
Ending0: When you are anxious, your pulse will go in the opposite direction while at rest. There are several common occurrences which signal heightened blood pressure (tachycardia).
Ending1: You may notice a shift in the way your heart rate lights up. This is called a desensitization.
Ending2: When you are anxious about a performance, your heart rate can become elevated as you tend to perform exceptionally fast or tense. [substeps] Any heart rate that is elevated immediately determines how fast your nerves function.
Ending3: You may hear your heat pounding in your ears if you are anxious enough. [substeps] This may also cause you to blush or your body temperature to rise.
Answer format: Ending0/Ending1/Ending2/Ending3"""},
]



generated_text = generator(messages,  max_length=500, num_return_sequences=1)

for i, text in enumerate(generated_text):
    print(f"Generated text {i + 1}:")
    text = text['generated_text']
    for entry in text:
        if entry['role'] == 'assistant':
            print(entry['content'])

Generated text 1:
Ending2: When you are anxious about a performance, your heart rate can become elevated as you tend to perform exceptionally fast or tense.


In [10]:
messages = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 18 Feb 2025

Please choose the correct ending to complete the given sentence. Follow the answer format, full answer not needed.<|eot_id|><|start_header_id|>user<|end_header_id|>

Health. [header] How to identify performance anxiety [title] Determine if your pulse is elevated. [step] When you are anxious about a performance, your pulse will become elevated. This is your body's response to the stress and worry of your impending performance.
Ending0: When you are anxious, your pulse will go in the opposite direction while at rest. There are several common occurrences which signal heightened blood pressure (tachycardia).
Ending1: You may notice a shift in the way your heart rate lights up. This is called a desensitization.
Ending2: When you are anxious about a performance, your heart rate can become elevated as you tend to perform exceptionally fast or tense. [substeps] Any heart rate that is elevated immediately determines how fast your nerves function.
Ending3: You may hear your heat pounding in your ears if you are anxious enough. [substeps] This may also cause you to blush or your body temperature to rise.
Answer format: Ending0/Ending1/Ending2/Ending3<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""



generated_text = generator(messages,  max_length=500, num_return_sequences=10)

for i in range(10):
    n = len(generated_text[i]['generated_text'])
    print(generated_text[i])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'generated_text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 18 Feb 2025\n\nPlease choose the correct ending to complete the given sentence. Follow the answer format, full answer not needed.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHealth. [header] How to identify performance anxiety [title] Determine if your pulse is elevated. [step] When you are anxious about a performance, your pulse will become elevated. This is your body's response to the stress and worry of your impending performance.\nEnding0: When you are anxious, your pulse will go in the opposite direction while at rest. There are several common occurrences which signal heightened blood pressure (tachycardia).\nEnding1: You may notice a shift in the way your heart rate lights up. This is called a desensitization.\nEnding2: When you are anxious about a performance, your heart rate can become elevated as you tend to perform exceptionally fast o

In [ ]:
# model.save_pretrained('./logs/models/step1')
# tokenizer.save_pretrained('./logs/models/step1')

('./logs/models/step1/tokenizer_config.json',
 './logs/models/step1/special_tokens_map.json',
 './logs/models/step1/tokenizer.json')